In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------
# Helper functions for input
# ---------------------------
def get_float(prompt, default=None):
    while True:
        s = input(prompt)
        if s.strip() == "" and default is not None:
            return default
        try:
            return float(s)
        except ValueError:
            print("Invalid number. Please try again.")

def get_choice(prompt, choices, default=None):
    choices = [str(c).lower() for c in choices]
    while True:
        s = input(prompt)
        if s.strip() == "" and default is not None:
            return default.lower()
        if s.lower() in choices:
            return s.lower()
        print("Choose one of: " + ", ".join(choices))

### Step 1: Beam Properties

In [ ]:
print("=== Beam Properties ===")
L = get_float("Enter beam length L (m): ")
b_dim = get_float("Enter beam width b (m): ")
h_dim = get_float("Enter beam height h (m): ")

# For a rectangular cross section, moment of inertia:
I = (b_dim * h_dim**3) / 12
print(f"Calculated moment of inertia I = {I:.4f} m^4")

### Step 2: Support Conditions

In [ ]:
print("\n=== Support Conditions ===")
support1 = get_float("Enter position of support 1 (m, measured from left, default 0): ", default=0.0)
support2 = get_float(f"Enter position of support 2 (m, measured from left, default {L}): ", default=L)
if support1 == support2:
    raise ValueError("Supports cannot be at the same location.")
print(f"Supports assumed at x = {support1:.2f} m and x = {support2:.2f} m.")

### Step 3: Load Inputs

In [ ]:
print("\n=== Load Inputs ===")
num_loads = int(get_float("Enter number of loads to apply: "))

loads = []
for i in range(num_loads):
    print(f"\nLoad #{i+1}:")
    ltype = get_choice("  Enter load type ('point' or 'udl'): ", ['point', 'udl'])
    if ltype == 'point':
        P = get_float("    Enter point load magnitude (kN, positive for downward): ")
        pos_ref = get_choice("    Is the position measured from Left (L) or Right (R)? ", ['L', 'R'], default='L')
        pos_val = get_float("    Enter the load position (m): ")
        if pos_ref == 'r':
            pos = L - pos_val
        else:
            pos = pos_val
        loads.append({"type": "point", "magnitude": P, "position": pos})
    elif ltype == 'udl':
        w = get_float("    Enter UDL intensity (kN/m, positive for downward): ")
        start_udl = get_float("    Enter UDL start position (m from left): ")
        end_udl = get_float("    Enter UDL end position (m from left): ")
        if start_udl < 0 or end_udl > L or start_udl >= end_udl:
            print("    Invalid UDL limits. Skipping this load.")
            continue
        loads.append({"type": "udl", "intensity": w, "start": start_udl, "end": end_udl})

### Step 4: Compute Support Reactions

In [ ]:
total_load = 0.0
moment_total = 0.0

for load in loads:
    if load["type"] == "point":
        P = load["magnitude"]
        pos = load["position"]
        total_load += P
        moment_total += P * (pos - support1)
    elif load["type"] == "udl":
        w = load["intensity"]
        start_udl = load["start"]
        end_udl = load["end"]
        L_udl = end_udl - start_udl
        load_udl = w * L_udl
        total_load += load_udl
        centroid = (start_udl + end_udl) / 2.0
        moment_total += load_udl * (centroid - support1)

R2 = moment_total / (support2 - support1)
R1 = total_load - R2

print("\n--- Support Reactions ---")
print(f"Reaction at support 1 (x = {support1} m): {R1:.3f} kN upward")
print(f"Reaction at support 2 (x = {support2} m): {R2:.3f} kN upward")

### Step 5: Compute Shear Force and Bending Moment Diagrams

In [ ]:
num_points = 500
x_vals = np.linspace(0, L, num_points)
S = np.zeros_like(x_vals)  # Shear force array
M = np.zeros_like(x_vals)  # Bending moment array

# Contribution from support reactions (only R1):
for i, x in enumerate(x_vals):
    if x >= support1:
        S[i] += R1
        M[i] += R1 * (x - support1)

# Subtract contributions from loads:
for load in loads:
    if load["type"] == "point":
        P = load["magnitude"]
        pos = load["position"]
        for i, x in enumerate(x_vals):
            if x >= pos:
                S[i] -= P
                M[i] -= P * (x - pos)
    elif load["type"] == "udl":
        w = load["intensity"]
        start_udl = load["start"]
        end_udl = load["end"]
        for i, x in enumerate(x_vals):
            if x < start_udl:
                continue
            elif x <= end_udl:
                length_effective = x - start_udl
                load_effective = w * length_effective
                S[i] -= load_effective
                centroid = start_udl + length_effective / 2.0
                M[i] -= load_effective * (x - centroid)
            else:
                load_total = w * (end_udl - start_udl)
                S[i] -= load_total
                centroid = (start_udl + end_udl) / 2.0
                M[i] -= load_total * (x - centroid)

### Step 6: Plotting the Results

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

# Beam Diagram
axs[0].set_title("Beam Diagram with Loads and Supports")
axs[0].hlines(0, 0, L, colors='black', linewidth=4)
axs[0].plot(support1, 0, marker='v', markersize=12, color='blue')
axs[0].plot(support2, 0, marker='v', markersize=12, color='blue')
axs[0].annotate(f"Support @ {support1:.2f} m", (support1, 0), xytext=(support1, 0.5),
                arrowprops=dict(arrowstyle="->", color='blue'), ha='center')
axs[0].annotate(f"Support @ {support2:.2f} m", (support2, 0), xytext=(support2, 0.5),
                arrowprops=dict(arrowstyle="->", color='blue'), ha='center')

for load in loads:
    if load["type"] == "point":
        P = load["magnitude"]
        pos = load["position"]
        arrow_length = -0.5 if P > 0 else 0.5
        axs[0].arrow(pos, 0, 0, arrow_length, head_width=0.2, head_length=0.1, fc='red', ec='red')
        axs[0].text(pos, arrow_length - 0.2, f"{P} kN", color='red', ha='center')
    elif load["type"] == "udl":
        w = load["intensity"]
        start_udl = load["start"]
        end_udl = load["end"]
        rect_height = 0.3
        axs[0].add_patch(plt.Rectangle((start_udl, 0.1), end_udl - start_udl, rect_height,
                                       color='green', alpha=0.3))
        axs[0].text((start_udl+end_udl)/2, 0.1+rect_height+0.1, f"{w} kN/m", color='green', ha='center')

axs[0].set_ylim(-2, 2)
axs[0].set_xlim(0, L)
axs[0].set_ylabel("Beam (m)")
axs[0].grid(True)

# Shear Force Diagram
axs[1].set_title("Shear Force Diagram")
axs[1].hlines(0, 0, L, colors='black', linewidth=2)
axs[1].plot(x_vals, S, 'b-', linewidth=2)
axs[1].fill_between(x_vals, S, 0, where=S < 0, color='blue', alpha=0.3)
axs[1].fill_between(x_vals, S, 0, where=S > 0, color='blue', alpha=0.3)
axs[1].set_ylabel("Shear (kN)")
axs[1].grid(True)

# Bending Moment Diagram
axs[2].set_title("Bending Moment Diagram (Below the Beam)")
axs[2].hlines(0, 0, L, colors='black', linewidth=2)
BM_offset = -max(abs(M)) * 0.1 if max(abs(M)) != 0 else 0
axs[2].plot(x_vals, M + BM_offset, 'r-', linewidth=2)
axs[2].fill_between(x_vals, M + BM_offset, BM_offset, color='red', alpha=0.3)
axs[2].set_ylabel("Bending Moment (kN·m)")
axs[2].set_xlabel("x (m)")
axs[2].grid(True)

plt.tight_layout()
plt.show()

### Step 7: Print Final Results

In [ ]:
print("\n=== Final Results ===")
print(f"Left Support Reaction at x = {support1} m: {R1:.3f} kN upward")
print(f"Right Support Reaction at x = {support2} m: {R2:.3f} kN upward")